## Basics of network analysis

https://github.com/socialcomquant/summer-school-2022/blob/main/Day2_Tizzoni_Network_Analysis/2_network_analysis.ipynb

DataSet : https://snap.stanford.edu/data/ca-AstroPh.html

This is data of undirected graphs with the links representing co-authorship


Topics covered:
- 
- 
- 

In [17]:
import networkx as nx

In [44]:
import pandas as pd
import numpy as np

In [19]:
%matplotlib inline

In [20]:
path = './Data/ca-AstroPh.txt'

In [21]:
df = pd.read_csv(path, sep = '\t')
df.head()

,# FromNodeId,ToNodeId
0,84424,276
1,84424,1662
2,84424,5089
3,84424,6058
4,84424,6229


In [22]:
G = nx.Graph()

In [23]:
for i in list(range(0, len(df),1)):
    G.add_edge(df['# FromNodeId'][i],df['ToNodeId'][i])

In [25]:
len(G.nodes())

18772

In [26]:
len(G.edges())

198110

#### Components in connected graphs

In [54]:
nx.is_connected(G)
#This should be false if there are components

False

In [55]:
nx.number_connected_components(G)

290

In [49]:
Node_Len = []
for k in nx.connected_components(G):
    Node_Len = np.append(Node_Len, len(k))
#This would give length of subgraph, which would essentially be the number of nodes.

In [50]:
len(Node_Len)

290

In [57]:
max(Node_Len)

17903.0

In [56]:
Components = list(nx.connected_components(G))

In [59]:
pos = 0
for j in range(1,len(Components),1):
    
    if(len(Components[j]) == max(Node_Len)):
        pos = j

#There would be a better sorting algo


In [61]:
len(Components[pos])

17903

In [62]:
G_C_Max = G.subgraph(Components[pos])

In [66]:
len(G_C_Max.nodes())
nx.is_connected(G_C_Max)
#This subgraphs is connected. #Could also be the largest co-author community.
len(G) - len(G_C_Max)

869